In [ ]:
import re
import time
import tkinter as tk
from tkinter import ttk
import logging

class ImcModel:
    
    # message d'erreur de saisie
    ERROR_FORMAT =" Il faut un entier de 3 caracteres max :"
    # pour tester les valeurs des champs
    pattern = r'[0-2]?[0-9]?[0-9]$'
    
    def __init__(self,poids,taille):
        self.set_poids(poids)
        self.set_taille(taille)
    
    def get_poids(self) :
        """
            retourne le poids
            :param : 
            :return :
        """
        return self.__poids
    
    def set_poids(self, unpoids:int):
        """
            valide un poids avant d'en affecter ou de modifier la valeur
            :param : unpoids
            :return :
        """
    
        if re.fullmatch(self.pattern,str(unpoids)):
            self.__poids = unpoids
        else :
            raise ValueError(f' {self.ERROR_FORMAT} {unpoids} ')
            
    def get_taille(self):
        """
            retourne la taille
            :param : 
            :return :
        """
        return self.__taille
    
    def set_taille(self,unetaille:int):
        """
            valide une taille avant d'en affecter ou de modifier la valeur
            :param : unetaille
            :return :
        """ 
        if re.fullmatch(self.pattern,str(unetaille)):
            self.__taille = unetaille
        else :
            raise ValueError(f'{self.ERROR_FORMAT} {unetaille}')
        
    poids = property(get_poids, set_poids)
    taille = property(get_taille, set_taille)
    
    def calculer(self):
        """
            effectue le calcul de l'imc'
            :param :
            :return :
        """
        return self.poids/(self.taille**2)
    
    def sauvegarder(self):
        """
            je sauvegarde les valeurs entrées dans un fichier
            :param :
            :return :
        """
        with open('imc.txt', 'a') as f:
            f.write(f" date: {time.ctime(time.time())} -  poids : {self.poids} - taille : {self.taille}  \n")
        


class ImcView(ttk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        
        #creation de widget
        # label poids
        self.poids_label = ttk.Label(self, text = "Poids kg")
        self.poids_label.grid(row=1, column=0)
        
        #poids
        self.poids_var = tk.StringVar()
        self.poids_saisie = ttk.Entry(self, textvariable=self.poids_var, width=15)
        self.poids_saisie.grid(row=1, column=1, sticky=tk.NSEW) # ou NUMERIC
        
        #label message pour erreur de saisie
        self.message_label = ttk.Label(self, text='', foreground='red')
        self.message_label.grid(row=2, column=1, sticky=tk.W)
        
        #label taille
        self.taille_label = ttk.Label(self, text = "Taille cm")
        self.taille_label.grid(row=3, column=0)
        
        #taille
        self.taille_var = tk.StringVar()
        self.taille_saisie = ttk.Entry(self, textvariable=self.taille_var, width=15)
        self.taille_saisie.grid(row=3, column=1, sticky=tk.NSEW) # ou NUMERIC
        
        #label message pour la taille
        #self.message_taille_label = ttk.Label(self, text='', foreground='red')
        #self.message_taille_label.grid(row=4, column=1, sticky=tk.W)
        
        #label imc
        self.imc_label = ttk.Label(self, text="IMC")
        self.imc_label.grid(row=5, column=0)
        
        #label message pour la taille
        self.value_imc_label = ttk.Label(self, text='valeur', foreground='black')
        self.value_imc_label.grid(row=5, column=1, sticky=tk.W)
        
        #bouton calculer
        self.calculer_bouton = ttk.Button(self, text='Calculer', command= self.calculer_bouton_cliquer)
        self.calculer_bouton.grid(row=9, column=0, padx=10)
        
        #bouton quitter
        self.quitter_bouton = ttk.Button(self, text='Quitter', command= self.quitter_bouton_cliquer)
        self.quitter_bouton.grid(row=11, column=0, padx=10)
        
       
        #bouton RAZ
        self.raz_bouton = ttk.Button(self, text='RAZ', command= self.raz_bouton_cliquer)
        self.raz_bouton.grid(row=13, column=0, padx=10)
        
        #controller
        self.controller = None
        
    def set_controller(self,controller):
        """
            met à jour le controller
            :param controller:
            :return :
        """
        self.controller = controller
        
    def calculer_bouton_cliquer(self):
        """
            effectue le calcul une fois le bouton cliqué
            :param : 
            :return :
        """
        if self.controller:
            self.controller.calculer(self.poids_var.get(), self.taille_var.get())
    
    def quitter_bouton_cliquer(self):
        self.destroy()
    
    def show_message_error(self, message):
        self.message_label['text']= message
        self.message_label['foreground'] = 'red'
        self.message_label.after(3000,self.message_cacher)
        
        self.poids_saisie['foreground'] = 'red'
    
    def show_message_success(self, message):
        """
            Montre un message en cas de success
            :param message: 
            :return :
        """
        self.message_label['text']= message
        self.message_label['foreground'] = 'green'
        self.message_label.after(3000,self.message_cacher)
        
        self.poids_saisie['foreground'] = 'black'
        self.taille_saisie['foreground'] = 'black'
        
    def message_cacher(self):
        """
            efface les message
            :param : 
            :return :
        """
        self.message_label['text']=''
        
        
    def raz_bouton_cliquer(self):
        """
            efface le contenu du formulaire
            :param : 
            :return :
        """
        self.value_imc_label['text']='valeur'
        
        # efface les saisie
        self.taille_saisie.delete(0, 'end')
        self.poids_saisie.delete(0, 'end')
        self.poids_var.set('')
        self.taille_var.set('')

        

class ImcController:
    def __init__(self, model, vue):
        self.model = model
        self.vue = vue
        
    def calculer(self, unpoids, unetaille):
        """
            effectue le calcul de l'imc
        """
        try:
            self.model.poids = int(unpoids)
            self.model.taille = int(unetaille)
            
            # calcul de la valeur
            self.vue.value_imc_label['text'] = str(self.model.calculer())
            
            #sauvegarde
            self.model.sauvegarder()
            
            # affiche un message de success
            self.vue.show_message_success(f" valeurs correctes : poids = {unpoids} || taille  ={unetaille} ")
        except ValueError :
            self.vue.show_message_error(f'{self.model.ERROR_FORMAT} poids = {unpoids} ||  taille = {unetaille}')
            logging.exception(f'{self.model.ERROR_FORMAT} poids = {unpoids} ||  taille = {unetaille}')
            
        except IOError:
            logging.exception("ne peut lire ou sauvegarder dans le fichier de sauvegarde imc.txt")
            

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.title('IMC - Formulaire de Calcul')
        
        model = ImcModel(0,0)
        
        vue = ImcView(self)
        vue.grid(row=0, column=0, padx=20, pady=20)
        
        controller = ImcController(model, vue)

        vue.set_controller(controller)

        
def main():
    
    logging.basicConfig(filename='imclog.log', level=logging.INFO, format='%(asctime)s : %(message)s')
    logging.info('Started')
    app = App()
    app.mainloop()
    logging.info('Finished')

if __name__ == '__main__':
    main()
        
        